# Complete delta tables guide

## Creating a delta table

Delta lake is the default format for table creation in notebooks, so there's no need to specify  
which kind of table will be created.
If an error emerges, you can use USING DELTA specification.

In [0]:
%sql

CREATE OR REPLACE TABLE employees
-- USING DELTA
( id INT, name STRING, salary DOUBLE )

In [0]:
%sql
INSERT INTO employees (id, name, salary) VALUES
(1, 'John Doe', 50000.00),
(2, 'Jane Smith', 60000.50),
(3, 'Peter Jones', 75000.75),
(4, 'Alice Brown', 55000.25),
(5, 'Bob Williams', 80000.00);

## Time traveling

History of a delta table can be accessed by using DESCRIBE HISTORY SQL Command

In [0]:
%sql

describe history employees;

Time travel queries (using timestamp as of)

In [0]:
%sql

select * 
from employees
Timestamp as of "2025-04-15 18:37:49.0";


Using a version number 

In [0]:
%sql
-- With an statement

select * 
from employees
version as of 3;

In [0]:
%sql

-- With the table name
select * 
from employees@v3;

Restoring versions of the table

In [0]:
%sql

restore table employees to version as of 3;

## Compacting files to improve reading performance

In [0]:
%sql
optimize employees

Compacting the table by ID

In [0]:
%sql

optimize employees
zorder by id;

## Droping versions of a table to clean up unused files

In [0]:
%sql

vacuum employees
-- default retention period is 7 days.

## Creating schemas/databases

Create the schema under the default directory (without specifying folder direction)

In [0]:
%sql 

create schema bronze_layer

In [0]:
%sql

describe extended employees;

Create the schema with a customized directory

In [0]:
%sql

create schema workspace.external_data

## External tables vs managed tables

Managed tables are stored in the database directory. Dropping the table will also drop the underlying data.

In [0]:
%sql
USE schema bronze_layer;

CREATE TABLE IF NOT EXISTS smartphones
(id INT, name STRING, brand STRING, year INT);

INSERT INTO smartphones
VALUES (1, 'iPhone 14', 'Apple', 2022),
      (2, 'iPhone 13', 'Apple', 2021),
      (3, 'iPhone 6', 'Apple', 2014),
      (4, 'iPad Air', 'Apple', 2013),
      (5, 'Galaxy S22', 'Samsung', 2022),
      (6, 'Galaxy Z Fold', 'Samsung', 2022),
      (7, 'Galaxy S9', 'Samsung', 2016),
      (8, '12 Pro', 'Xiaomi', 2022),
      (9, 'Redmi 11T Pro', 'Xiaomi', 2022),
      (10, 'Redmi Note 11', 'Xiaomi', 2021)

External tables are created outside the database directory. Dropping the table would not delete the underlying data.

In [0]:
%sql

create or replace table hive_metastore.default.apple_smartphones_1
comment "Just apple smartphones"
partitioned by (id,year)
location "dbfs:/external_data/delta_tables"
as 
select * 
from workspace.bronze_layer.smartphones 
where brand = 'Apple';

## Table constraints

Not null constraints

In [0]:
%sql
ALTER TABLE bronze_layer.apple_smartphones 
ALTER COLUMN id SET NOT NULL;

Check constraints

In [0]:
%sql
ALTER TABLE bronze_layer.smartphones 
ADD CONSTRAINT valid_year CHECK (year > 2010);

## Clone tables

Deep clone: Data+Metadata

In [0]:
%sql

create table if not exists bronze_layer.smartphones_copy
deep clone bronze_layer.smartphones;

Shallow clone: just metadata.


In [0]:
%sql
create table if not exists bronze_layer.smartphones_shallow_copy
shallow clone bronze_layer.smartphones;

## Views

Normal views

In [0]:
%sql
use schema bronze_layer;

create view if not exists xiaomi_smartphones as
select * 
from smartphones
where brand = 'Xiaomi'
;

In [0]:
%sql
select * from bronze_layer.xiaomi_smartphones

Temporary views (session-scoped)



In [0]:
%sql
use schema bronze_layer;

create temp view xiaomi_smartphones_old as
select * 
from smartphones
where brand = 'Xiaomi' and year < 2022
;


Global temporary view 

(Cluster-scoped view)

View dropped when cluster restarted.

In [0]:
%sql
use schema bronze_layer;

create global temp view xiaomi_smartphones_new as
select * 
from smartphones
where brand = 'Xiaomi' and year > 2022
;